In [2]:
import pandas as pd
from census import Census
from us import states
from shapely.geometry import Point
import geopandas as gp
from scipy.stats import ttest_ind
import rtree
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 250)

In [8]:
df = pd.read_csv('../other_data/2016_census_tract_data.csv', dtype={'geoid': str})
df.head()

,geoid,name,total_population,median_income,median_home_value,educational_attainment,white_alone,black_alone,native,asian,native_hawaiian_pacific_islander,some_other_race_alone,two_or_more,hispanic_or_latino,unemployed,labor_force,receives_snap,snap_universe,receives_pai,pai_universe,uninsured,insured_universe,median_percentage_income_rent
0,06001400100,"Census Tract 4001, Alameda County, California",3018,177417.0,1074100.0,1981.0,2145.0,92.0,0.0,491.0,0.0,75.0,132.0,83.0,75.0,1643.0,0.0,1292.0,9.0,1292.0,110.0,3018.0,17.6
1,06001400200,"Census Tract 4002, Alameda County, California",1960,153125.0,978900.0,1278.0,1426.0,11.0,4.0,165.0,0.0,5.0,138.0,211.0,41.0,1270.0,5.0,813.0,0.0,813.0,73.0,1960.0,18.5
2,06001400300,"Census Tract 4003, Alameda County, California",5236,85313.0,912700.0,2817.0,3346.0,606.0,9.0,599.0,36.0,0.0,254.0,386.0,204.0,3402.0,55.0,2439.0,58.0,2439.0,160.0,5236.0,27.0
3,06001400400,"Census Tract 4004, Alameda County, California",4171,99539.0,848900.0,2512.0,2758.0,410.0,25.0,458.0,0.0,0.0,266.0,266.0,114.0,2678.0,29.0,1798.0,27.0,1798.0,173.0,4165.0,23.7
4,06001400500,"Census Tract 4005, Alameda County, California",3748,83650.0,683500.0,1822.0,1972.0,944.0,0.0,140.0,24.0,19.0,163.0,486.0,82.0,2545.0,50.0,1643.0,71.0,1643.0,397.0,3748.0,24.3


In [9]:
df = df.rename(index=str, columns={"geoid": "GEOID"})

In [5]:
all_tracts = (
    gp.read_file(
        "../other_data/cb_2016_06_tract_500k.json"
    )
)
all_tracts.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,06,013,359103,1400000US06013359103,06013359103,3591.03,CT,2593523,1170833,"POLYGON ((-122.302287 37.995278, -122.298516 3..."
1,06,013,366001,1400000US06013366001,06013366001,3660.01,CT,1017166,0,"POLYGON ((-122.353307 37.978544, -122.338327 3..."
2,06,013,374000,1400000US06013374000,06013374000,3740,CT,992095,0,"POLYGON ((-122.347471 37.943629, -122.334702 3..."
3,06,013,388000,1400000US06013388000,06013388000,3880,CT,659857,0,"POLYGON ((-122.308489 37.911273, -122.302276 3..."
4,06,017,030710,1400000US06017030710,06017030710,307.10,CT,7898800,0,"POLYGON ((-121.081663 38.692493, -121.057808 3..."


In [10]:
merged = all_tracts.merge(df, on='GEOID')
merged.shape

(1582, 32)